# Hand Gesture Volume Control

In [ ]:
import cv2
import numpy as np
import mediapipe as mp   # pip install mediapipe
from math import hypot

from ctypes import cast,POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface=devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL,None)
volume=cast(interface,POINTER(IAudioEndpointVolume))


min_vol,max_vol=volume.GetVolumeRange()[:2]

print("min",min_vol)
print("max",max_vol)
mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw= mp.solutions.drawing_utils

cap=cv2.VideoCapture(0)

while True:
    success,img=cap.read()
    
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results= hands.process(imgRGB)
    
    lmList= []
    if results.multi_hand_landmarks:
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark):
                
                
                h,w,_= img.shape
                cx,cy= int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy])
                mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)
                
    if lmList !=[]:
        x1,y1= lmList[4][1],lmList[4][2]
        x2,y2= lmList[8][1],lmList[8][2]
        
        cv2.circle(img,(x1,y1),4,(255,0,0),cv2.FILLED)
        cv2.circle(img,(x1,y1),4,(255,0,0),cv2.FILLED)
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)
         
        length= hypot(x2-x1,y2-y1)
        
        vol=np.interp(length,[25,300],[min_vol,max_vol])     # min distance=25 & max distance=300
        
        volume.SetMasterVolumeLevel(vol, None)
        
    
    cv2.imshow('Image',img)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()



min -63.5
max 0.0
